In [2]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sympy.abc import epsilon
from sympy.functions.special.hyper import HyperRep

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
df.shape

(569, 33)

In [5]:
df.drop(columns= ['id','Unnamed: 32'],inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [7]:
X = df.drop(columns=['diagnosis'])
y = df['diagnosis']

In [8]:
X_train,X_test, y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
scaler = StandardScaler()


In [10]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [11]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

## Converting np array to tensors for NN


In [12]:
X_train_tensor = torch.from_numpy(X_train_scaled)
X_test_tensor = torch.from_numpy(X_test_scaled)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)


In [13]:
X_train_tensor.shape

torch.Size([455, 30])

## Defining the model
n

In [14]:
class mysinpleNN():
    def __init__(self,X):
        self.weights = torch.rand(X.shape[1],1,dtype=torch.float64,
                                  requires_grad=True)
        self.bias = torch.zeros(1,dtype=torch.float64,requires_grad=True)


    def forward(self,X):
        z = torch.matmul(X,self.weights)+self.bias
        y_pred = torch.sigmoid(z)
        return y_pred

    def loss_function(self, y_pred, y):
        # Clamp predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        # Calculate loss
        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss


## Imp parameters


In [18]:
learning_rate = 0.1
epochs = 25


### Training Pipeline

# Steps to make a model
- create model
- define loop
- forward pass
- loss calc
- backward pass
- parameter update


In [22]:
# create a model
model = mysinpleNN(X_train_tensor)
# model.weights
# model.bias
for epochs in range(epochs):
    y_pred = model.forward(X_train_tensor) # forward pass
    # print(y_pred)


## loss calc
    loss = model.loss_function(y_pred,y_train_tensor)
    print(f'epochs : {epochs},Loss : {loss.item()}')

    ## backward pass
    loss.backward()

    ## update parameters
    with torch.no_grad():
        model.weights -= learning_rate* model.weights.grad
        model.bias -= learning_rate* model.bias.grad


    #zero gradients

    model.weights.grad.zero_()
    model.bias.grad.zero_()

    #print loss each epoch

    print(f"f Epoch : {epochs+1} Loss : {loss.item()}")

epochs : 0,Loss : 3.139830443199588
f Epoch : 1 Loss : 3.139830443199588
epochs : 1,Loss : 3.007747368372583
f Epoch : 2 Loss : 3.007747368372583
epochs : 2,Loss : 2.8681843350603007
f Epoch : 3 Loss : 2.8681843350603007
epochs : 3,Loss : 2.7266467423401983
f Epoch : 4 Loss : 2.7266467423401983
epochs : 4,Loss : 2.5853614615295117
f Epoch : 5 Loss : 2.5853614615295117
epochs : 5,Loss : 2.4482266066918514
f Epoch : 6 Loss : 2.4482266066918514
epochs : 6,Loss : 2.3158216932211495
f Epoch : 7 Loss : 2.3158216932211495
epochs : 7,Loss : 2.1876595777765147
f Epoch : 8 Loss : 2.1876595777765147
epochs : 8,Loss : 2.0600049550819883
f Epoch : 9 Loss : 2.0600049550819883
epochs : 9,Loss : 1.9314752341505899
f Epoch : 10 Loss : 1.9314752341505899
epochs : 10,Loss : 1.8057744934889728
f Epoch : 11 Loss : 1.8057744934889728
epochs : 11,Loss : 1.685215410342116
f Epoch : 12 Loss : 1.685215410342116
epochs : 12,Loss : 1.572958859400824
f Epoch : 13 Loss : 1.572958859400824
epochs : 13,Loss : 1.46817

# Model Evaluation


In [26]:
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)
    y_pred = (y_pred>0.9).float()
    accuracy = (y_pred==y_test_tensor).float().mean()
    print(f'Accuracy {accuracy.item()}')


print(y_pred)

Accuracy 0.6012619137763977
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
      